In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%pylab
from src import data

from src import efficient_frontier
from src import filter_assets
from src import plot
from src import portfolio
import numpy as np
import pandas as pd
import ipywidgets
from scipy.optimize import Bounds

Using matplotlib backend: TkAgg
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1100
max_esg_score = 2000
df = pd.read_excel(r"C:\Users\toros\OneDrive\Dokumenter\GitHub\BachelorThesis\data\ESG_DATA_S&P500.xlsx")

dates = ['2013-02-04','2015-01-12']
start_year = dates[0]
end_year = dates[1]

Bounds = Bounds(0,1) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.02

In [7]:
esg_data = data.esg_score_weight(df,weights,min_esg_score,max_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data)



1 Failed download:
- CTVA: Data doesn't exist for startDate = 1364788800, endDate = 1448946000

1 Failed download:
- FTV: Data doesn't exist for startDate = 1364788800, endDate = 1448946000

1 Failed download:
- HWM: Data doesn't exist for startDate = 1364788800, endDate = 1448946000

1 Failed download:
- INVH: Data doesn't exist for startDate = 1364788800, endDate = 1448946000

1 Failed download:
- MRNA: Data doesn't exist for startDate = 1364788800, endDate = 1448946000

1 Failed download:
- OGN: Data doesn't exist for startDate = 1364788800, endDate = 1448946000


In [9]:
prices = prices[['MMM','AOS','ABT','ABBV','ACN','ATVI','ADM','ADBE','ADP','AAP']]
esgdata = esgdata[['MMM','AOS','ABT','ABBV','ACN','ATVI','ADM','ADBE','ADP','AAP']]

prices

KeyError: ('MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ATVI', 'ADM', 'ADBE', 'ADP', 'AAP')

In [ ]:
 #Gives us price data and esg data to use in future functions
pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds, sharpe_type,start_year,end_year, Wanted_return, maximum_risk) #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters) #Gives us the weights of the individual stocks in our portfolio
portfolio_esg_score = portfolio.esg_score_of_portfolio(weights_of_each_portfolio,esgdata.head(1)) #gives us the ESG score of our portfolio
portfolio_allocations = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[1] #How much we allocate to our market portfolio compared to the risk-free asset
cmle_returns = portfolio.capital_mark_line_returns(parameters,risk_free_rate,maximum_risk)[0] #Return based on how much we allocate to the risk free asset.


In [ ]:
weights_of_each_portfolio

In [ ]:
portfolio_allocations

In [ ]:
weights_of_each_portfolio.max()

In [ ]:
portfolio_esg_score

In [ ]:
cmle_returns

In [ ]:
#plot.plot_efficient_frontier_cml(parameters,start_year,end_year,risk_free_rate)


ipywidgets.interactive(plot.plot_efficient_frontier_cml, parameters = ipywidgets.fixed(parameters),
                       start_year = ipywidgets.fixed(start_year),
                       end_year = ipywidgets.fixed(end_year),
                       risk_free_rate=ipywidgets.fixed(risk_free_rate),
                       plot_cml = True,
                       risk = (0,1,0.01),
                       mpl_style='default',
                       title='Efficient Frontier')


#def plot_efficient_frontier_cml(parameters,start_year,end_year,risk_free_rate,plot_cml = True,
#                            risk = 0.2,
#                            mpl_style='default',
#                            title='Efficient Frontier')